Helpers: 
* https://stackoverflow.com/questions/32173053/copy-columns-from-workbook-paste-in-second-sheet-of-second-workbook-openpyxl
* https://automatetheboringstuff.com/chapter12/

In [1]:
import sys, os, openpyxl, xlrd
from openpyxl.utils import get_column_letter
from copy import copy
import pandas as pd
import numpy as np

In [2]:
#Get the filename
dataDir = '../../Data'
dataFN = dataDir + os.sep + 'LouisianaWaterUse_Distribute.xlsx'
templateFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet.xlsx'
outFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet2.xlsx'

In [3]:
#Read the data xlsx file in as a dataframe
dfRaw = pd.read_excel(dataFN,sheetname='data1')

In [4]:
#Select only 2000, 2005, and 2010 records and list the number of data for each column
dfSelect = dfRaw[dfRaw.Year.isin([2000,2005,2010])]

In [5]:
#Pivot on year, listing usage (Annual Amt. MGD) by Source Water  Type and SCI1 Description
dfPivot = dfSelect.pivot_table(columns='Year',
                               index=('SIC1','Description'),
                               values='Annual Amt. (MGD)',
                               aggfunc='sum')
#Reset the index to add index values back as columns
dfPivot.reset_index(inplace=True)

In [6]:
#Create a list of sector names and append a TOTAL entry
sectorNames = dfPivot.Description.values.tolist()
sectorNames.append("TOTAL")

In [7]:
#Open the balance sheet template workbook
wb = openpyxl.load_workbook(templateFN)

In [8]:
#Set the year
year = 2000

In [9]:
#Get the sheet for the given year
ws = wb.get_sheet_by_name(str(year))

In [10]:
def updateColumn(worksheet,col,name,useGW=0):
    '''Updates the specified column in the balance sheet.
       col = the letter of the column to be updated
       name = the name of the sector
       useGW = the use from groundwater
    '''
    #Sector names
    worksheet['{0}2'.format(col)].value = name
    worksheet['{0}19'.format(col)].value = name

    ##Supply calculations
    worksheet['{0}12'.format(col)].value = '=SUM({0}9:{0}11)'.format(col)
    worksheet['{0}16'.format(col)].value = '=SUM({0}14:{0}15)'.format(col)
    worksheet['{0}17'.format(col)].value = '=SUM({0}12,{0}16)'.format(col)

    ##Use calculations
    worksheet['{0}24'.format(col)].value = '=SUM({0}21:{0}23)'.format(col)
    worksheet['{0}29'.format(col)].value = '=SUM({0}26:{0}28)'.format(col)
    worksheet['{0}34'.format(col)].value = '=SUM({0}24,{0}29)'.format(col)

    ##Totals
    worksheet['{0}36'.format(col)].value = '=({0}34-{0}17)'.format(col)
    
    ##Values
    worksheet['{0}22'.format(col)].value = useGW


In [11]:
#Create totals column
def updateTotalCol(worksheet, colNum):
    totalCol = get_column_letter(colNum)
    prevCol = get_column_letter(colNum - 1)

    #Add sum formula in the following rows
    for row in [4,5,6,7,9,10,11,12,14,15,16,17,20,21,22,23,24,26,27,28,29,31,32,32,34]:
        worksheet['{0}{1}'.format(totalCol,row)].value = '=SUM(B{0}:{1}{0})'.format(row,prevCol)

    #Add the grand total (supply - use)
    worksheet['{0}36'.format(totalCol)].value = '=({0}34-{0}17)'.format(totalCol)

In [12]:
#Update the cells of the first sector column
col = 'B'
name = sectorNames[0]
usage = dfPivot.loc[dfPivot.Description == sectorNames[0], year].iloc[0]

updateColumn(ws, col, name, usage)

In [13]:
#Loop through each sector, by index
for idx in range(1,len(sectorNames)):
    #get the sector name at the index
    sectorName = sectorNames[idx]
    #get the letter of the column to create
    colLetter = get_column_letter(idx+1)
    
    #Add a new column and copy the style
    for src, dst in zip(ws['B:B'], ws['{0}:{0}'.format(colLetter)]):
        if src.value: dst.value = str(src.value).replace('B',colLetter)
        if src.has_style:
            dst.font = copy(src.font)
            dst.fill = copy(src.fill)
            dst.number_format = copy(src.number_format)
            dst.alignment = copy(src.alignment)
            dst.border = copy(src.border)
            
    #Update the value for the current sector, unless it's the TOTAL column
    if sectorName != 'TOTAL':
        usage = dfPivot.loc[dfPivot.Description == sectorName, 2000].iloc[0]
        updateColumn(ws, colLetter, sectorName, usage)
        
    #Otherwise, fill the column with the formulae for the Totals
    updateTotalCol(ws, idx+1)


In [14]:
#Save
wb.save(outFN)